Twitter Sentiment Analysis

Import the libraries that will be usedin this project.

In [83]:
import pandas as pd
import numpy as np

pd.options.display.max_colwidth = 1000

Read in the twitter data into a panda dataframe. I originally ran into an encoding issue and had to save the csv in UTF-8. The csv file does not include a column header row, so add those in manually.

In [84]:
df = pd.read_csv("twitter_data.csv", header=None, names=["sentiment", "tweet_id", "date", "query", "user", "tweet"])

In [85]:
df.head()

,sentiment,tweet_id,date,query,user,tweet
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by texting it... and might cry as a result School today also. Blah!
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Managed to save 50% The rest go out of bounds
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all. i'm mad. why am i here? because I can't see you all over there."


Remove columns that dont seem useful for sentiment analysis

In [86]:
df.drop(["tweet_id", "date", "query", "user"], axis=1, inplace=True)
df.head()

,sentiment,tweet
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
1,0,is upset that he can't update his Facebook by texting it... and might cry as a result School today also. Blah!
2,0,@Kenichan I dived many times for the ball. Managed to save 50% The rest go out of bounds
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all. i'm mad. why am i here? because I can't see you all over there."


DO NOT KEEP THIS IN THE PROJECT!!!
DROPPING A BUNCH OF ROWS TO SPEED UP PREPROCESSING WHILE TESTING!!!

In [87]:
df = df.sample(1000)
df.shape

(1000, 2)

Perform data preprocessing

In [88]:
import re

def processTweet(tweet):
    tweet = re.sub("[@|#]\w+\S","", tweet) # remove @usernames and #hashtags
    tweet = re.sub("http[s]?://[\S]+", '', tweet) # remove urls 
    tweet = re.sub(r"(.)\1\1+",r"\1\1", tweet) # remove letters that repeat more than 2 times
    return tweet

In [89]:
df['tweet'] = df['tweet'].map(lambda tweet: processTweet(tweet))

df.head()

,sentiment,tweet
982927,4,"cant be bothered going to the gym, having a glass on wine instead!!"
421141,0,what about your UK fans
1506952,4,helping Peter Facinelli win his bet. HELP HIM TOO. Just start to follow him here o Twitter
1518112,4,and you love noobs
1339468,4,happy birthday guys! x


In [90]:
from sklearn.model_selection import train_test_split

y = df['sentiment']
X = df.drop(['sentiment'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.25, random_state=42)

In [91]:
from sklearn.feature_extraction.text import CountVectorizer

count_vector = CountVectorizer()

train_vector = count_vector.fit_transform(X_train['tweet'])
test_vector = count_vector.transform(X_test['tweet'])

In [92]:
from sklearn.naive_bayes import MultinomialNB
naive_bayes = MultinomialNB()
naive_bayes.fit(train_vector, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [93]:
predictions = naive_bayes.predict(test_vector)

In [94]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y_test,predictions))
print(classification_report(y_test,predictions))
print(accuracy_score(y_test, predictions))

[[91 24]
 [54 81]]
              precision    recall  f1-score   support

           0       0.63      0.79      0.70       115
           4       0.77      0.60      0.68       135

    accuracy                           0.69       250
   macro avg       0.70      0.70      0.69       250
weighted avg       0.71      0.69      0.69       250

0.688
